In [3]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip uninstall -y torch

!pip install torch==2.8.0
!pip install torch_geometric
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.8.0+cu126.html
!pip install pyg-lib -f https://data.pyg.org/whl/torch-2.8.0+cu126.html
# !pip install faiss-cpu

Found existing installation: torch 2.10.0+cu128
Uninstalling torch-2.10.0+cu128:
  Successfully uninstalled torch-2.10.0+cu128
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.4/322.4 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.6/155.6 MB 9.6 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.6.0
    Uninstalling triton-3.6.0:
      Successfully uninstalled triton-3.6.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.27.5
    Uninstalling nvidia-nccl-cu12-2.27.5:
      Successfully uninstalled nvidia-nccl-cu12-2.27.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.10.0+cu128 requires torch==2.10.0, but you have torch 2.8.0 which is incompatible.
torchvision 0.2

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

import logging

from torch_geometric.data import HeteroData
from torch_geometric.loader import NeighborLoader
from torch_geometric.transforms import ToUndirected
from torch_geometric.nn import SAGEConv, HeteroConv, GATv2Conv, GraphNorm

import logging

import numpy as np
import os
import pandas as pd
import math

import gc

import matplotlib.pyplot as plt

import plotly.express as px
from google.colab import drive
import re

import json
import pickle

import plotly.graph_objects as go

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, cohen_kappa_score
from sklearn.model_selection import train_test_split

import sys, os
sys.path.append(os.path.abspath(".."))


import os
import json
import copy
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import HeteroConv, GATv2Conv, GraphNorm, SAGEConv, HGTConv

## Functions

In [5]:
class QuantileLoss(nn.Module):
    def __init__(self, quantile=0.5):
        super().__init__()
        self.quantile = quantile

    def forward(self, preds, target):
        errors = target - preds
        return torch.max(
            (self.quantile - 1) * errors,
            self.quantile * errors
        ).mean()


class WeightedMSELoss(nn.Module):
    def __init__(self, threshold=10.0, high_weight=2.0):
        super().__init__()
        self.threshold = threshold
        self.high_weight = high_weight

    def forward(self, preds, target):
        weights = torch.ones_like(target)
        weights[target > self.threshold] = self.high_weight
        return (weights * (preds - target) ** 2).mean()


def get_loss_fn(loss_type="mse"):
    """
    loss_type options:
    - 'mse'
    - 'mae'
    - 'huber'
    - 'quantile'
    - 'weighted'
    """
    if loss_type == "mse":
        return nn.MSELoss()
    elif loss_type == "mae":
        return nn.L1Loss()
    elif loss_type == "huber":
        return nn.SmoothL1Loss(beta=1.0)
    elif loss_type == "quantile":
        return QuantileLoss(quantile=0.5)
    elif loss_type == "weighted":
        return WeightedMSELoss(threshold=10.0, high_weight=3.0)
    else:
        raise ValueError(f"Unknown loss type: {loss_type}")

def save_hyperparameters(hyperparameters, model_name, directory ='/Models'):

  model_dir = f'{directory}/{model_name}/'
  if not os.path.exists(model_dir):
      os.makedirs(model_dir)

  hyperparameters_path = os.path.join(model_dir, f'hyperparameters.json')

  with open(hyperparameters_path, 'w') as f:
      json.dump(hyperparameters, f)

In [6]:
class HeteroGNN(nn.Module):
  def __init__(self, data, metadata, hidden_channels=64, out_channels=1,
                conv_layer=GATv2Conv, dropout=0.3):
      super().__init__()
      self.hidden_channels = hidden_channels
      self.dropout = nn.Dropout(dropout)
      self.conv_layer = conv_layer

      self.lin_dict = nn.ModuleDict()
      self.res_lin_dict = nn.ModuleDict()
      self.norms = nn.ModuleDict()
      self.embeddings = nn.ModuleDict()

      for ntype in metadata[0]:
          if getattr(data[ntype], 'x', None) is not None:
              in_channels = data[ntype].x.size(1)
              self.lin_dict[ntype] = nn.Linear(in_channels, hidden_channels)
              self.res_lin_dict[ntype] = nn.Linear(in_channels, hidden_channels)
              self.norms[ntype] = GraphNorm(hidden_channels)
          else:
              self.embeddings[ntype] = nn.Embedding(data[ntype].num_nodes, hidden_channels)
              self.norms[ntype] = nn.Identity()

      self.conv1 = HeteroConv({
          et: conv_layer((-1, -1), hidden_channels, add_self_loops=False)
          for et in metadata[1]
      }, aggr='mean')
      self.conv2 = HeteroConv({
          et: conv_layer((-1, -1), hidden_channels, add_self_loops=False)
          for et in metadata[1]
      }, aggr='mean')

      self.reg_head = nn.Linear(hidden_channels, out_channels)

  def forward(self, x_dict, edge_index_dict, batch_dict=None):
        device = next(self.parameters()).device
        x_processed = {}

        for ntype in x_dict:
            x = x_dict[ntype]
            if x is None:
                x = self.embeddings[ntype](
                    torch.arange(self.embeddings[ntype].num_embeddings, device=device)
                )
            else:
                h = self.lin_dict[ntype](x)
                res = self.res_lin_dict[ntype](x)
                x = F.relu(h + res)

            norm = self.norms[ntype]
            if isinstance(norm, GraphNorm) and batch_dict is not None and ntype in batch_dict:
                x = norm(x, batch_dict[ntype])
            else:
                x = norm(x)

            x_processed[ntype] = self.dropout(x)

        filtered_ei1 = {et: edge_index_dict[et]
                        for et in self.conv1.convs.keys()
                        if et in edge_index_dict}

        out_conv1 = self.conv1(x_processed, filtered_ei1)

        x_current_layer = {}
        for ntype in x_processed.keys():
            h = out_conv1.get(ntype, x_processed[ntype])
            h = F.relu(h)
            h = self.dropout(h)
            x_current_layer[ntype] = h


        filtered_ei2 = {et: edge_index_dict[et]
                        for et in self.conv2.convs.keys()
                        if et in edge_index_dict}

        out_conv2 = self.conv2(x_current_layer, filtered_ei2)

        x_final = {}
        for ntype in x_processed.keys():
            h = out_conv2.get(ntype, x_current_layer[ntype])

            if ntype in x_processed:
                h = h + x_processed[ntype]

            x_final[ntype] = self.dropout(F.relu(h))

        return self.reg_head(x_final['stay'])


In [19]:
def train(model, loader, optimizer, loss_fn, device='cuda'):
    model.train()
    total_loss = 0

    seed_bs = loader.batch_size

    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()

        pred = model(batch.x_dict, batch.edge_index_dict)

        pred = pred[:seed_bs]
        target = batch['stay'].y[:seed_bs].unsqueeze(1)

        loss = loss_fn(pred, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)

import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, cohen_kappa_score

BINS = [0,1,2,3,5,7,10,14,21,30,np.inf]

def msle(y_true, y_pred):
    return np.mean((np.log1p(y_true) - np.log1p(y_pred))**2)

def mad(y_true, y_pred):
    return np.median(np.abs(y_true - y_pred))

def log_mape(y_true, y_pred):
    return np.mean(np.abs((np.log1p(y_true) - np.log1p(y_pred)) / np.log1p(y_true))) * 100

def kappa_linear(y_true, y_pred):
    yb = np.digitize(y_true, BINS)
    pb = np.digitize(y_pred, BINS)
    return cohen_kappa_score(yb, pb, weights="linear")


@torch.no_grad()
def evaluate(model, loader, loss_fn, device='cuda'):
    model.eval()

    all_preds, all_targets = [], []
    total_loss = 0

    for batch in loader:
        batch = batch.to(device)
        seed_bs = loader.batch_size

        pred = model(batch.x_dict, batch.edge_index_dict)
        pred = pred[:seed_bs]
        target = batch['stay'].y[:seed_bs].unsqueeze(1).to(device)

        loss = loss_fn(pred, target)
        total_loss += loss.item()

        all_preds.append(pred.cpu().numpy())
        all_targets.append(target.cpu().numpy())

    preds = np.concatenate(all_preds).flatten()
    targets = np.concatenate(all_targets).flatten()


    # Metrics calculated on log-transformed values
    r2 = r2_score(targets, preds)
    val_loss = total_loss / len(loader)
    mae_log = mean_absolute_error(targets, preds)
    mse_log = mean_squared_error(targets, preds)

    # Convert back to original scale for specific metrics
    true_orig = np.expm1(targets)
    pred_orig = np.expm1(preds)

    msle_val = msle(true_orig, pred_orig)
    mad_val = mad(true_orig, pred_orig)
    lmape = log_mape(true_orig, pred_orig)
    kappa = kappa_linear(true_orig, pred_orig)

    return (
        val_loss,
        r2,
        mae_log,
        mse_log,
        msle_val,
        mad_val,
        lmape,
        kappa,
        preds,
        targets
    )



def train_loop(model, train_loader, val_loader, optimizer, loss_fn, epochs,
               patience, model_dir, device='cuda', plot_rate=10, start_epoch= 0):

    best_val_loss = float('inf')
    counter = 0

    for epoch in range(start_epoch + 1, epochs + start_epoch + 1):
        train_loss = train(model, train_loader, optimizer, loss_fn, device)
        val_loss, r2, mae_log, mse_log, msle, mad, lmape, kappa, preds, targets= evaluate(model, val_loader, loss_fn, device)

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        val_r2_scores.append(r2)
        val_mae_scores.append(mae_log)
        val_mse_scores.append(mse_log)
        val_msle_scores.append(msle)
        val_mad_scores.append(mad)
        val_lmape_scores.append(lmape)
        val_kappa_scores.append(kappa)

        print(
                f"Epoch {epoch} | "
                f"Loss: {val_loss:.4f} | "
                f"R2(log): {r2:.4f} | "
                f"MAE(log): {mae_log:.3f} | "
                f"MSE(log): {mse_log:.3f} | "
                f"MSLE: {msle:.3f} | "
                f"MAD: {mad:.3f} | "
                f"logMAPE: {lmape:.2f}% | "
                f"Kappa: {kappa:.3f}"
                )


        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            torch.save(model.state_dict(), f'{model_dir}')

        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping after {epoch} epochs due to no improvement in validation loss.")
                torch.save(model.state_dict(), f'{model_dir[:-3]}_best.pt')

                with open(f'{model_dir[:-8]}targets.pkl', 'wb') as f:
                  pickle.dump(targets, f)

                with open(f'{model_dir[:-8]}peredicts.pkl', 'wb') as f:
                  pickle.dump(preds, f)
                break
        torch.cuda.empty_cache()
        gc.collect()

        if epoch % plot_rate == 0 or epoch == 5:

            fig1 = go.Figure()

            fig1.add_trace(go.Scatter(
                x=list(range(1, len(train_losses) + 1)),
                y=train_losses,
                mode='lines',
                name='Train Loss'
            ))

            fig1.add_trace(go.Scatter(
                x=list(range(1, len(val_losses) + 1)),
                y=val_losses,
                mode='lines',
                name='Validation Loss'
            ))

            fig1.update_layout(
                title='Training and Validation Loss over Epochs',
                xaxis_title='Epoch',
                yaxis_title='Loss',
                template='plotly_white'
            )

            fig1.show()

            fig2 = go.Figure()

            fig2.add_trace(go.Scatter(
                x=list(range(1, len(val_r2_scores) + 1)),
                y=val_r2_scores,
                mode='lines',
                name='Validation R2 (log-space)'
            ))

            fig2.add_trace(go.Scatter(
                x=list(range(1, len(val_mae_scores) + 1)),
                y=val_mae_scores,
                mode='lines',
                name='MAE (log-space)'
            ))

            fig2.add_trace(go.Scatter(
                x=list(range(1, len(val_mse_scores) + 1)),
                y=val_mse_scores,
                mode='lines',
                name='MSE (log-space)'
            ))

            fig2.add_trace(go.Scatter(
                x=list(range(1, len(val_msle_scores) + 1)),
                y=val_msle_scores,
                mode='lines',
                name='MSLE'
            ))

            fig2.add_trace(go.Scatter(
                x=list(range(1, len(val_mad_scores) + 1)),
                y=val_mad_scores,
                mode='lines',
                name='MAD (days)'
            ))

            fig2.add_trace(go.Scatter(
                x=list(range(1, len(val_lmape_scores) + 1)),
                y=val_lmape_scores,
                mode='lines',
                name='log-MAPE (%)'
            ))

            fig2.add_trace(go.Scatter(
                x=list(range(1, len(val_kappa_scores) + 1)),
                y=val_kappa_scores,
                mode='lines',
                name='Kappa'
            ))

            fig2.update_layout(
                title='Validation Metrics over Epochs',
                xaxis_title='Epoch',
                yaxis_title='Metric Value',
                template='plotly_white'
            )

            fig2.show()


            with open(f'{model_dir[:-8]}results.json', 'wb') as f:
                pickle.dump({
                    'train_losses': train_losses,
                    'val_losses': val_losses,
                    'val_r2_scores': val_r2_scores,
                    'val_mae_scores': val_mae_scores,
                    'val_mse_scores': val_mse_scores,
                    'val_msle_scores': val_msle_scores,
                    'val_mad_scores': val_mad_scores,
                    'val_lmape_scores': val_lmape_scores,
                    'val_kappa_scores': val_kappa_scores,
                    'best_val_loss': best_val_loss,
                }, f)


            with open(f'{model_dir[:-8]}targets.pkl', 'wb') as f:
                pickle.dump(targets, f)

            with open(f'{model_dir[:-8]}peredicts.pkl', 'wb') as f:
                pickle.dump(preds, f)


    return (
        train_losses,
        val_losses,
        val_r2_scores,
        best_val_loss,
        val_mae_scores,
        val_mse_scores,
        val_msle_scores,
        val_mad_scores,
        val_lmape_scores,
        val_kappa_scores
    )

def get_conv_layer(conv_layer):
    if conv_layer == 'SAGEConv':
        return SAGEConv
    elif conv_layer == 'GATv2Conv':
        return GATv2Conv
    elif conv_layer == 'HGTConv':
        return HGTConv
    else:
        raise ValueError(f"Unknown convolution layer: {conv_layer}")


In [22]:
def make_transductive_loaders(
    data,
    fanouts=(15, 10),
    batch_size=2048,
    num_workers=4,
    seed=42,):

    torch.manual_seed(seed)

    train_idx = data["stay"].train_mask.nonzero(as_tuple=False).view(-1)
    val_idx   = data["stay"].val_mask.nonzero(as_tuple=False).view(-1)
    test_idx  = data["stay"].test_mask.nonzero(as_tuple=False).view(-1)

    num_neighbors = list(fanouts)

    train_loader = NeighborLoader(
        data,
        input_nodes=("stay", train_idx),
        num_neighbors=num_neighbors,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        persistent_workers=(num_workers > 0),
    )

    val_loader = NeighborLoader(
        data,
        input_nodes=("stay", val_idx),
        num_neighbors=num_neighbors,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        persistent_workers=(num_workers > 0),
    )

    test_loader = NeighborLoader(
        data,
        input_nodes=("stay", test_idx),
        num_neighbors=num_neighbors,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        persistent_workers=(num_workers > 0),
    )

    return train_loader, val_loader, test_loader

## Run

In [13]:
dir = '/content/drive/MyDrive/Thesis/Codes/Data'
save_file_address = f'Graph_v2_1'
data = torch.load(f'{dir}/{save_file_address}/data_chart.pt', weights_only=False)

print(data['stay'].y.max())
data['stay'].y = torch.log1p(data['stay'].y)
print(data['stay'].y.max())


tensor(226.4031)
tensor(5.4267)


In [14]:
import datetime

hyperparameters = {
    'hidden_channels': 256,
    'conv_layer': 'GATv2Conv',
    'dropout': 0.3,
    'lr': 1e-3,
    'loss': 'huber',
    'data' : 'v2_chart',
    'similarity': 'non',
    'model_name': 'simple',
    'num_neighbors' : (15, 10)
}

metadata = data.metadata()



model_dir = f'{dir}/{save_file_address}/Models'

timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')

model_name = f"model_{hyperparameters['data']}_{hyperparameters['similarity']}_{timestamp}"

save_hyperparameters(hyperparameters, model_name, directory = model_dir)


train_loader, val_loader, test_loader = make_transductive_loaders(
    data,
    fanouts=hyperparameters['num_neighbors'],
    batch_size=4096,
    num_workers=4,
    seed=42
)


device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = HeteroGNN(data,
                  metadata,
                  hidden_channels=hyperparameters['hidden_channels'],
                  out_channels=1,
                  conv_layer=get_conv_layer(hyperparameters['conv_layer']),
                  dropout = hyperparameters['dropout']).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr= hyperparameters['lr'])
loss_fn = get_loss_fn(hyperparameters['loss'])


train_losses, val_losses, val_r2_scores, val_mae_scores, val_mse_scores = [], [], [], [], []
val_msle_scores, val_mad_scores, val_lmape_scores, val_kappa_scores = [], [], [], []


# model.load_state_dict(torch.load(f'{model_dir}/{model_name}/model.pt'))
# model.eval()

# with open(f'{model_dir}/{model_name}/results.json', 'rb') as f:
#     result = pickle.load(f)

# train_losses = result['train_losses']
# val_losses = result['val_losses']
# val_r2_scores = result['val_r2_scores']
# val_mae_scores = result['val_mae_scores']
start_epoch = len(train_losses)

total_epochs = 150
best_val_loss = float('inf')
epochs = total_epochs - start_epoch

patience = 50
counter = 0

train_loader, val_loader, test_loader = make_transductive_loaders(
    data,
    fanouts=hyperparameters['num_neighbors'],
    batch_size=4096,
    num_workers=4,
    seed=42
)

train_losses, val_losses, val_r2_scores, best_val_loss, val_mae_scores, val_mse_scores, val_msle_scores, val_mad_scores, val_lmape_scores, val_kappa_scores = train_loop(model, train_loader, val_loader,
                   optimizer, loss_fn, epochs,
                   patience, f'{model_dir}/{model_name}/model.pt',
                   device, plot_rate=50 ,start_epoch= start_epoch)

/tmp/ipython-input-6697/2605668847.py:24: UserWarning:

There exist node types ({'patient'}) whose representations do not get updated during message passing as they do not occur as destination type in any edge type. This may lead to unexpected behavior.

/tmp/ipython-input-6697/2605668847.py:28: UserWarning:

There exist node types ({'patient'}) whose representations do not get updated during message passing as they do not occur as destination type in any edge type. This may lead to unexpected behavior.



Epoch 1 | Loss: 0.2010 | R2(log): 0.0518 | MAE: 2.433 | MSE: 31.607 | MSLE: 0.432 | MAD: 1.022 | logMAPE: 56.32% | Kappa: 0.220
Epoch 2 | Loss: 0.2112 | R2(log): 0.0343 | MAE: 2.392 | MSE: 31.755 | MSLE: 0.440 | MAD: 0.930 | logMAPE: 49.43% | Kappa: 0.218
Epoch 3 | Loss: 0.2120 | R2(log): 0.0457 | MAE: 2.355 | MSE: 31.262 | MSLE: 0.435 | MAD: 0.881 | logMAPE: 47.13% | Kappa: 0.221
Epoch 4 | Loss: 0.2077 | R2(log): 0.0692 | MAE: 2.352 | MSE: 31.778 | MSLE: 0.424 | MAD: 0.878 | logMAPE: 46.04% | Kappa: 0.242
Epoch 5 | Loss: 0.2132 | R2(log): 0.0440 | MAE: 2.346 | MSE: 31.162 | MSLE: 0.435 | MAD: 0.858 | logMAPE: 44.46% | Kappa: 0.238


Epoch 6 | Loss: 0.2075 | R2(log): 0.0744 | MAE: 2.332 | MSE: 32.131 | MSLE: 0.422 | MAD: 0.837 | logMAPE: 42.65% | Kappa: 0.252
Epoch 7 | Loss: 0.2066 | R2(log): 0.0738 | MAE: 2.311 | MSE: 31.298 | MSLE: 0.422 | MAD: 0.815 | logMAPE: 39.75% | Kappa: 0.256
Epoch 8 | Loss: 0.2023 | R2(log): 0.0940 | MAE: 2.285 | MSE: 30.632 | MSLE: 0.413 | MAD: 0.808 | logMAPE: 38.42% | Kappa: 0.274
Epoch 9 | Loss: 0.1938 | R2(log): 0.1385 | MAE: 2.246 | MSE: 30.442 | MSLE: 0.392 | MAD: 0.751 | logMAPE: 36.44% | Kappa: 0.294
Epoch 10 | Loss: 0.1876 | R2(log): 0.1632 | MAE: 2.228 | MSE: 29.847 | MSLE: 0.381 | MAD: 0.773 | logMAPE: 36.79% | Kappa: 0.309
Epoch 11 | Loss: 0.1945 | R2(log): 0.1292 | MAE: 2.293 | MSE: 35.293 | MSLE: 0.397 | MAD: 0.764 | logMAPE: 37.20% | Kappa: 0.298
Epoch 12 | Loss: 0.1836 | R2(log): 0.1849 | MAE: 2.219 | MSE: 30.770 | MSLE: 0.371 | MAD: 0.738 | logMAPE: 35.51% | Kappa: 0.319
Epoch 13 | Loss: 0.1767 | R2(log): 0.2226 | MAE: 2.180 | MSE: 29.436 | MSLE: 0.354 | MAD: 0.718 | log

Epoch 51 | Loss: 0.1508 | R2(log): 0.3436 | MAE: 2.055 | MSE: 27.860 | MSLE: 0.299 | MAD: 0.637 | logMAPE: 31.41% | Kappa: 0.394
Epoch 52 | Loss: 0.1453 | R2(log): 0.3665 | MAE: 2.040 | MSE: 27.472 | MSLE: 0.289 | MAD: 0.652 | logMAPE: 31.90% | Kappa: 0.400
Epoch 53 | Loss: 0.1438 | R2(log): 0.3727 | MAE: 2.037 | MSE: 27.691 | MSLE: 0.286 | MAD: 0.643 | logMAPE: 31.40% | Kappa: 0.394
Epoch 54 | Loss: 0.1436 | R2(log): 0.3738 | MAE: 2.033 | MSE: 27.729 | MSLE: 0.285 | MAD: 0.647 | logMAPE: 30.92% | Kappa: 0.398
Epoch 55 | Loss: 0.1429 | R2(log): 0.3778 | MAE: 2.030 | MSE: 27.498 | MSLE: 0.283 | MAD: 0.634 | logMAPE: 30.97% | Kappa: 0.395
Epoch 56 | Loss: 0.1469 | R2(log): 0.3537 | MAE: 2.052 | MSE: 27.523 | MSLE: 0.294 | MAD: 0.654 | logMAPE: 30.81% | Kappa: 0.404
Epoch 57 | Loss: 0.1404 | R2(log): 0.3840 | MAE: 2.030 | MSE: 26.692 | MSLE: 0.281 | MAD: 0.663 | logMAPE: 31.89% | Kappa: 0.416
Epoch 58 | Loss: 0.1457 | R2(log): 0.3653 | MAE: 2.042 | MSE: 27.800 | MSLE: 0.289 | MAD: 0.631 |

Epoch 101 | Loss: 0.1393 | R2(log): 0.3928 | MAE: 2.019 | MSE: 27.277 | MSLE: 0.277 | MAD: 0.636 | logMAPE: 30.43% | Kappa: 0.403
Epoch 102 | Loss: 0.1363 | R2(log): 0.4074 | MAE: 2.010 | MSE: 27.126 | MSLE: 0.270 | MAD: 0.653 | logMAPE: 30.56% | Kappa: 0.401
Epoch 103 | Loss: 0.1343 | R2(log): 0.4160 | MAE: 2.010 | MSE: 27.020 | MSLE: 0.266 | MAD: 0.676 | logMAPE: 31.45% | Kappa: 0.402
Epoch 104 | Loss: 0.1352 | R2(log): 0.4093 | MAE: 2.005 | MSE: 26.843 | MSLE: 0.269 | MAD: 0.649 | logMAPE: 30.47% | Kappa: 0.409
Epoch 105 | Loss: 0.1313 | R2(log): 0.4272 | MAE: 1.998 | MSE: 26.434 | MSLE: 0.261 | MAD: 0.679 | logMAPE: 30.69% | Kappa: 0.417
Epoch 106 | Loss: 0.1288 | R2(log): 0.4379 | MAE: 1.995 | MSE: 26.268 | MSLE: 0.256 | MAD: 0.707 | logMAPE: 31.23% | Kappa: 0.415
Epoch 107 | Loss: 0.1337 | R2(log): 0.4161 | MAE: 2.005 | MSE: 26.752 | MSLE: 0.266 | MAD: 0.668 | logMAPE: 30.66% | Kappa: 0.413
Epoch 108 | Loss: 0.1340 | R2(log): 0.4168 | MAE: 2.009 | MSE: 26.834 | MSLE: 0.266 | MAD:

In [17]:
import warnings
warnings.filterwarnings("ignore")

In [30]:
dir = '/content/drive/MyDrive/Thesis/Codes/Data'
save_file_address = f'Graph_v2_1'
data = torch.load(f'{dir}/{save_file_address}/data.pt', weights_only=False)

In [31]:
print(data['stay'].y.max())
data['stay'].y = torch.log1p(data['stay'].y)
print(data['stay'].y.max())


hyperparameters = {
    'hidden_channels': 256,
    'conv_layer': 'GATv2Conv',
    'dropout': 0.3,
    'lr': 1e-3,
    'loss': 'huber',
    'data' : 'v2',
    'similarity': 'non',
    'model_name': 'simple',
    'num_neighbors' : (15, 10)
}

metadata = data.metadata()


model_dir = f'{dir}/{save_file_address}/Models'

timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')

model_name = f"model_{hyperparameters['data']}_{hyperparameters['similarity']}_{timestamp}"

save_hyperparameters(hyperparameters, model_name, directory = model_dir)


train_loader, val_loader, test_loader = make_transductive_loaders(
    data,
    fanouts=hyperparameters['num_neighbors'],
    batch_size=4096,
    num_workers=4,
    seed=12
)


device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = HeteroGNN(data,
                  metadata,
                  hidden_channels=hyperparameters['hidden_channels'],
                  out_channels=1,
                  conv_layer=get_conv_layer(hyperparameters['conv_layer']),
                  dropout = hyperparameters['dropout']).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr= hyperparameters['lr'])
loss_fn = get_loss_fn(hyperparameters['loss'])


train_losses, val_losses, val_r2_scores, val_mae_scores, val_mse_scores = [], [], [], [], []
val_msle_scores, val_mad_scores, val_lmape_scores, val_kappa_scores = [], [], [], []


# model.load_state_dict(torch.load(f'{model_dir}/{model_name}/model.pt'))
# model.eval()

# with open(f'{model_dir}/{model_name}/results.json', 'rb') as f:
#     result = pickle.load(f)

# train_losses = result['train_losses']
# val_losses = result['val_losses']
# val_r2_scores = result['val_r2_scores']
# val_mae_scores = result['val_mae_scores']
start_epoch = len(train_losses)

total_epochs = 150
best_val_loss = float('inf')
epochs = total_epochs - start_epoch

patience = 50
counter = 0

train_loader, val_loader, test_loader = make_transductive_loaders(
    data,
    fanouts=hyperparameters['num_neighbors'],
    batch_size=4096,
    num_workers=4,
    seed=42
)

train_losses, val_losses, val_r2_scores, best_val_loss, val_mae_scores, val_mse_scores, val_msle_scores, val_mad_scores, val_lmape_scores, val_kappa_scores = train_loop(model, train_loader, val_loader,
                   optimizer, loss_fn, epochs,
                   patience, f'{model_dir}/{model_name}/model.pt',
                   device, plot_rate=50 ,start_epoch= start_epoch)

tensor(226.4031)
tensor(5.4267)
Epoch 1 | Loss: 0.0980 | R2(log): 0.5456 | MAE(log): 0.319 | MSE(log): 0.207 | MSLE: 0.207 | MAD: 0.683 | logMAPE: 35.68% | Kappa: 0.635
Epoch 2 | Loss: 0.0675 | R2(log): 0.7338 | MAE(log): 0.254 | MSE(log): 0.121 | MSLE: 0.121 | MAD: 0.556 | logMAPE: 27.23% | Kappa: 0.693
Epoch 3 | Loss: 0.0689 | R2(log): 0.7382 | MAE(log): 0.267 | MSE(log): 0.119 | MSLE: 0.119 | MAD: 0.605 | logMAPE: 28.18% | Kappa: 0.687
Epoch 4 | Loss: 0.0712 | R2(log): 0.7349 | MAE(log): 0.274 | MSE(log): 0.121 | MSLE: 0.121 | MAD: 0.636 | logMAPE: 28.32% | Kappa: 0.675
Epoch 5 | Loss: 0.0765 | R2(log): 0.7129 | MAE(log): 0.290 | MSE(log): 0.131 | MSLE: 0.131 | MAD: 0.685 | logMAPE: 30.25% | Kappa: 0.654


Epoch 6 | Loss: 0.0744 | R2(log): 0.7205 | MAE(log): 0.287 | MSE(log): 0.127 | MSLE: 0.127 | MAD: 0.675 | logMAPE: 30.72% | Kappa: 0.660
Epoch 7 | Loss: 0.0743 | R2(log): 0.7239 | MAE(log): 0.287 | MSE(log): 0.126 | MSLE: 0.126 | MAD: 0.670 | logMAPE: 30.45% | Kappa: 0.657
Epoch 8 | Loss: 0.0713 | R2(log): 0.7338 | MAE(log): 0.281 | MSE(log): 0.121 | MSLE: 0.121 | MAD: 0.645 | logMAPE: 31.29% | Kappa: 0.668
Epoch 9 | Loss: 0.0638 | R2(log): 0.7662 | MAE(log): 0.263 | MSE(log): 0.106 | MSLE: 0.106 | MAD: 0.613 | logMAPE: 28.57% | Kappa: 0.685
Epoch 10 | Loss: 0.0545 | R2(log): 0.7960 | MAE(log): 0.233 | MSE(log): 0.093 | MSLE: 0.093 | MAD: 0.531 | logMAPE: 24.95% | Kappa: 0.723
Epoch 11 | Loss: 0.0501 | R2(log): 0.8212 | MAE(log): 0.223 | MSE(log): 0.081 | MSLE: 0.081 | MAD: 0.512 | logMAPE: 23.77% | Kappa: 0.729
Epoch 12 | Loss: 0.0450 | R2(log): 0.8363 | MAE(log): 0.202 | MSE(log): 0.075 | MSLE: 0.075 | MAD: 0.458 | logMAPE: 21.04% | Kappa: 0.752
Epoch 13 | Loss: 0.0418 | R2(log): 0.8

Epoch 51 | Loss: 0.0210 | R2(log): 0.9369 | MAE(log): 0.102 | MSE(log): 0.029 | MSLE: 0.029 | MAD: 0.179 | logMAPE: 9.76% | Kappa: 0.870
Epoch 52 | Loss: 0.0284 | R2(log): 0.9154 | MAE(log): 0.151 | MSE(log): 0.039 | MSLE: 0.039 | MAD: 0.323 | logMAPE: 16.73% | Kappa: 0.811
Epoch 53 | Loss: 0.0226 | R2(log): 0.9364 | MAE(log): 0.116 | MSE(log): 0.029 | MSLE: 0.029 | MAD: 0.233 | logMAPE: 11.27% | Kappa: 0.845
Epoch 54 | Loss: 0.0199 | R2(log): 0.9339 | MAE(log): 0.093 | MSE(log): 0.030 | MSLE: 0.030 | MAD: 0.138 | logMAPE: 8.55% | Kappa: 0.886
Epoch 55 | Loss: 0.0189 | R2(log): 0.9441 | MAE(log): 0.098 | MSE(log): 0.025 | MSLE: 0.025 | MAD: 0.166 | logMAPE: 10.26% | Kappa: 0.879
Epoch 56 | Loss: 0.0190 | R2(log): 0.9405 | MAE(log): 0.100 | MSE(log): 0.027 | MSLE: 0.027 | MAD: 0.171 | logMAPE: 10.69% | Kappa: 0.881
Epoch 57 | Loss: 0.0179 | R2(log): 0.9477 | MAE(log): 0.086 | MSE(log): 0.024 | MSLE: 0.024 | MAD: 0.129 | logMAPE: 7.89% | Kappa: 0.889
Epoch 58 | Loss: 0.0215 | R2(log): 0.

Epoch 101 | Loss: 0.0134 | R2(log): 0.9560 | MAE(log): 0.069 | MSE(log): 0.020 | MSLE: 0.020 | MAD: 0.079 | logMAPE: 6.04% | Kappa: 0.910
Epoch 102 | Loss: 0.0141 | R2(log): 0.9633 | MAE(log): 0.072 | MSE(log): 0.017 | MSLE: 0.017 | MAD: 0.097 | logMAPE: 6.60% | Kappa: 0.904
Epoch 103 | Loss: 0.0173 | R2(log): 0.9541 | MAE(log): 0.098 | MSE(log): 0.021 | MSLE: 0.021 | MAD: 0.172 | logMAPE: 10.71% | Kappa: 0.874
Epoch 104 | Loss: 0.0144 | R2(log): 0.9616 | MAE(log): 0.067 | MSE(log): 0.017 | MSLE: 0.017 | MAD: 0.078 | logMAPE: 6.08% | Kappa: 0.911
Epoch 105 | Loss: 0.0146 | R2(log): 0.9629 | MAE(log): 0.071 | MSE(log): 0.017 | MSLE: 0.017 | MAD: 0.098 | logMAPE: 6.50% | Kappa: 0.904
Epoch 106 | Loss: 0.0123 | R2(log): 0.9619 | MAE(log): 0.067 | MSE(log): 0.017 | MSLE: 0.017 | MAD: 0.087 | logMAPE: 6.22% | Kappa: 0.910
Epoch 107 | Loss: 0.0129 | R2(log): 0.9548 | MAE(log): 0.072 | MSE(log): 0.021 | MSLE: 0.021 | MAD: 0.095 | logMAPE: 6.34% | Kappa: 0.906
Epoch 108 | Loss: 0.0147 | R2(log

In [32]:
(  _,
    r2,
    mae_log,
    mse_log,
    msle_val,
    mad_val,
    lmape,
    kappa,
    preds,
    targets
    ) = evaluate(model, test_loader, loss_fn, device='cuda')

with open(f'{model_dir}/{model_name}/results.json', 'w') as f:
    json.dump({
        'test_r2_scores': float(r2),
        'test_mae_scores': float(mae_log),
        'test_mse_scores': float(mse_log),
        'test_msle_scores': float(msle_val),
        'test_mad_scores': float(mad_val),
        'test_lmape_scores': float(lmape),
        'test_kappa_scores': float(kappa)
    }, f)


with open(f'{model_dir}/{model_name}/targets.pkl', 'wb') as f:
    pickle.dump(targets, f)

with open(f'{model_dir}/{model_name}/peredicts.pkl', 'wb') as f:
    pickle.dump(preds, f)